In [1]:
import warnings
import time
import os
import pandas as pd

# From raw trajectory data to generate smoothed speed data

In [ ]:
from macro_data_processing import get_speed_mean_field
from macro_data_processing import convert_to_cst_unix
from ASM import smooth_raw_data

In [ ]:
# Declare your root directory where you have subdirectories named by date
#  following the convention on the data repository (e.g., 2022-11-21).
inception_root = ''

dirs = [dr for dr in os.listdir(inception_root) if dr[0] != '.']
# Iterate through each date subdirectory.
for date_dir in dirs:
    # Read the date as the subdirectory name, append times, convert to Unix timestamp.
    starttime = convert_to_cst_unix(date_dir + ' 06:00')
    endtime = convert_to_cst_unix(date_dir + ' 10:00')
    # Find all JSON files in the subdirectory.
    data_files = [fn for fn in os.listdir(os.path.join(inception_root, date_dir)) 
                  if os.path.splitext(fn)[1] == '.json']
    if len(data_files) == 0:
        print("SKIPPING DIRECTORY {}\n".format(date_dir))
        continue
    else:
        # There should only be one JSON data file in each directory.
        input_filename = data_files[0]
        input_file_path = os.path.join(inception_root, date_dir, input_filename)
    # Filename portion to which we will append additional naming.
    input_stub = os.path.splitext(input_filename)[0]
    print(f"\nWorking on new input file: {input_file_path}")
    print(f"Date of input file: {date_dir}")
    print(f"Time range {starttime} to {endtime}")
    
    # dx and dt are the size of Edie's box.
    dx = 0.02
    dt = 4

    # Compute the raw and smooth spatiotemporal speed fields for each lane 1-4.
    for lane_number in range(1, 5):
        speed_raw_output_file = os.path.join(inception_root, date_dir, input_stub + '_speed_raw_lane{}.csv'.format(lane_number))
        speed_smooth_output_file = os.path.join(inception_root, date_dir, input_stub + '_speed_smooth_lane{}.csv'.format(lane_number))
        print("Output file paths:")
        print(f"\tRaw speed: {speed_raw_output_file}")
        print(f"\tSmooth speed: {speed_smooth_output_file}")
        
        t0 = time.time()
        # Silence/ignore an expected divide by zero error.
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            speed_raw = get_speed_mean_field(input_file_path, dx, dt, starttime, endtime, lane_number)
        print(f"Speed field compute time: {time.time() - t0}s")
        
        print(f"Writing raw speed field to {speed_raw_output_file}")
        speed_raw.to_csv(speed_raw_output_file, index=False)
        
        t0 = time.time()
        smooth_speed = smooth_raw_data(speed_raw, dx, dt, 0.1, 24)
        print(f"Smoothing compute time: {time.time() - t0}s")
        smooth_speed.to_csv(speed_smooth_output_file, index=False)
    

# From smoothed data to generate virtual trajectories 

In [3]:
from macro_data_processing import convert_to_cst_unix
from virtual_trajectory_generation import gen_all_VT

In [ ]:
inception_root = ''
dirs = [dr for dr in os.listdir(inception_root) if dr[0] != '.']
for date_dir in dirs:
    starttime = convert_to_cst_unix(date_dir + ' 06:00')
    endtime = convert_to_cst_unix(date_dir + ' 10:00')
    data_files = [fn for fn in os.listdir(os.path.join(inception_root, date_dir)) 
                  if os.path.splitext(fn)[1] == '.csv' and 'smooth' in fn]
    if len(data_files) == 0:
        print("SKIPPING DIRECTORY {}\n".format(date_dir))
        continue
    for lane_data_file in data_files:
        lane_number = int(os.path.splitext(lane_data_file)[0].split('_')[-1][-1])
        input_file_path = os.path.join(inception_root, date_dir, lane_data_file)
        print(f"\nWorking on new input file: {input_file_path}")
        print(f"Date of input file: {date_dir}")
        print(f"Time range {starttime} to {endtime}")
        contains_dot_underscore = "._" in input_file_path
        if contains_dot_underscore==False:
            smooth_speed = pd.read_csv(input_file_path)
            vt_all = gen_all_VT(smooth_speed, frequency = 120, hour=3)
            vt_all.to_csv(os.path.join(inception_root, date_dir) + f'/vt_lane{lane_number}.csv', index=False)
            print(f"Virtual trajectories for {date_dir} lane {lane_number} is generated.")


Working on new input file: /Volumes/WorkLabJJ/Data/I-24 MOTION PUBLIC DATASETS/VT-tools_INCEPTION_v1/2022-11-21/637b023440527bf2daa5932f__post1_speed_smooth_lane1.csv
Date of input file: 2022-11-21
Time range 1669032000 to 1669046400
Virtual trajectories for 2022-11-21 lane 1 is generated.

Working on new input file: /Volumes/WorkLabJJ/Data/I-24 MOTION PUBLIC DATASETS/VT-tools_INCEPTION_v1/2022-11-21/637b023440527bf2daa5932f__post1_speed_smooth_lane2.csv
Date of input file: 2022-11-21
Time range 1669032000 to 1669046400
